# Sources

- Fruits-360 Dataset [Link](https://github.com/Horea94/Fruit-Images-Dataset)

## Model Architecture
Define and create our models Architecture

In [ ]:
import random
import pandas as pd
import numpy as np
import torchvision
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

#Load Data
transforms_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
image_data_train = ImageFolder("./Training", transform=transforms_train) #Specify Training Data Dir
image_data_test = ImageFolder("./Training", transform=transforms_train) #Specify Validation Image Dir

#Shuffle data to train properly
random.shuffle(image_data_train.samples)
random.shuffle(image_data_test.samples)

#Total Classes
classes_idx = image_data_train.class_to_idx
classes = len(image_data_train.classes)
len_train_data = len(image_data_train)
len_test_data = len(image_data_test)

def get_labels():
    labels_train = []
    labels_test = []

    for i in image_data_train.imgs:
        labels_train.append(i[1])

    for j in image_data_test.imgs:
        labels_test.append(j[1])

    return (labels_train, labels_test)

labels_train, labels_test = get_labels()

train_loader = DataLoader(dataset=image_data_train, batch_size=100)
test_loader = DataLoader(dataset=image_data_test, batch_size=100)

print(next(iter(train_loader))[0].shape)
record = next(iter(train_loader))[0]

#Flatten Layer
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

#Main Model
class Model:
    def build_model(self):
        model = nn.Sequential(nn.Conv2d(3, 64, kernel_size=5, stride=1), nn.ReLU(), 
                              nn.MaxPool2d(2), nn.Conv2d(64, 64, kernel_size=7, stride=1),
                              nn.ReLU(), nn.MaxPool2d(3), nn.Conv2d(64, 64, kernel_size=7),
                              nn.ReLU(), nn.MaxPool2d(5), Flatten(), nn.Linear(64, 100), 
                              nn.ReLU(), nn.Linear(100, 65))
        return model
    
model = Model()
model = model.build_model()

#print(model)

The above code outputs

### Model Architecture

In summary,  
~~~py 
torch.Size([100, 3, 100, 100])
~~~

Indicates the shape of the images loaded into the batch. The first 100 is the batch size indicating how many images are in a batch. The 3 indicates the amound of colour channels in each image within the batch. In this instance all the images are using RGB. The two 100s are the image dimensions with all of Fruits-360 being 100x100 images 

~~~py
Sequential(
(0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
(1): ReLU()
(2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
(3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
(4): ReLU()
(5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
(6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
(7): ReLU()
(8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
(9): Flatten()
(10): Linear(in_features=64, out_features=100, bias=True)
(11): ReLU()
(12): Linear(in_features=100, out_features=65, bias=True)
)
~~~

Descibes the architecture of each part of the model. Below is the architecture in raw.

RELU (Rectified Linear Unit) is an activation function used in neural networks that outputs the input directly if it is positive; otherwise, it outputs zero. It is used here as ReLU introduces non-linearity to the model, enabling it to learn complex patterns and relationships within the data.

(0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1)): A 2D convolutional layer that takes 3 input channels (RGB) and outputs 64 feature maps using a 5x5 kernel with a stride of 1.

(1): ReLU(): A ReLU (Rectified Linear Unit) activation function, which introduces non-linearity to the model.

(2): MaxPool2d(kernel_size=2, stride=2): A max-pooling layer with a 2x2 kernel and a stride of 2, which downsamples the feature maps by taking the maximum value in each 2x2 window.

(3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1)): Another 2D convolutional layer that takes 64 input channels and outputs 64 feature maps using a 7x7 kernel with a stride of 1.

(4): ReLU(): Another ReLU activation function.

(5): MaxPool2d(kernel_size=3, stride=3): A max-pooling layer with a 3x3 kernel and a stride of 3, further downsampling the feature maps.

(6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1)): A third 2D convolutional layer that maintains 64 input and output channels using a 7x7 kernel with a stride of 1.

(7): ReLU(): Another ReLU activation function.

(8): MaxPool2d(kernel_size=5, stride=5): A max-pooling layer with a 5x5 kernel and a stride of 5, further downsampling the feature maps.

(9): Flatten(): A custom flatten layer that reshapes the multi-dimensional tensor into a 2D tensor (batch size, -1) to prepare it for the fully connected layers.

(10): Linear(in_features=64, out_features=100): A fully connected (linear) layer that takes 64 input features and outputs 100 features.

(11): ReLU(): Another ReLU activation function.

(12): Linear(in_features=100, out_features=65): The final fully connected layer that takes 100 input features and outputs 65 features, corresponding to the number of classes (65 classes).

### Moving on

If the line "print(labels_train[0])" is run the following is output

As seen above there once again the batch size and attributes and 12 is printed. The 12 is the first image in the batch  
If the line "print (image_data_train.samples[0][1])" is run the following is returned.

This is once again the first Tuple in the batch  
Next is the training and deployment of the model

## Training

To deploy our NN we need to first train it based on our dataset. This can be done with the code below. To do this the fucntion train is created. This can be seen down below

In [ ]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01) # Learn Rate And Optimizer
criterion = nn.CrossEntropyLoss() # Set Loss Type

#Train the Model
def train(epochs):
    model.train()
    losses = []
    for epoch in range(1, epochs+1):
        print(f"Epoch #{epoch}")
        current_loss = 0
        for feature, label in train_loader:
            x = Variable(feature, requires_grad=False).float()
            y = Variable(label, requires_grad=False).long()

            optimizer.zero_grad()
            y_pred = model(x) # Zero the grads
            correct = y_pred.max(1)[1].eq(y).sum()
            print(f"Number of correct classifications: {correct.item()}")
            
            loss = criterion(y_pred, y)
            print(f"Loss: {loss.item()}")
            current_loss += loss.item()
            loss.backward() # Calc Gradient
            optimizer.step() # Change Weights
        losses.append(current_loss) # Log Loss after Epoch Completion
    return losses

Epochs are effectivly a training session in which the model is trained and tested. 10s or 100s of epochs are required for accurate predictions.  
  
However once run the following is returned.

This can be easily solved by setting the correct ammount of output classes in the final layer of the model this can be done by replacing the lines: 

In [ ]:
class Model:
    def build_model(self):
        model = nn.Sequential(nn.Conv2d(3, 64, kernel_size=5, stride=1), nn.ReLU(), 
                              nn.MaxPool2d(2), nn.Conv2d(64, 64, kernel_size=7, stride=1),
                              nn.ReLU(), nn.MaxPool2d(3), nn.Conv2d(64, 64, kernel_size=7),
                              nn.ReLU(), nn.MaxPool2d(5), Flatten(), nn.Linear(64, 100), 
                              nn.ReLU(), nn.Linear(100, 65))
        return model

With

In [ ]:
num_classes = len(image_data_train.classes)

class Model:
    def build_model(self):
        model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1), nn.ReLU(),
            nn.MaxPool2d(2), nn.Conv2d(64, 64, kernel_size=7, stride=1),
            nn.ReLU(), nn.MaxPool2d(3), nn.Conv2d(64, 64, kernel_size=7),
            nn.ReLU(), nn.MaxPool2d(5), Flatten(), nn.Linear(64, 100),
            nn.ReLU(), nn.Linear(100, num_classes)  # Updated to use correct number of classes
        )
        return model

This means that the amount of classes in the final layer will be updated dynamically

## Testing

Now we need to test our models accuracy against the images that are used for validation. This can be done by implementing the below code:

In [ ]:
def test():
    model.eval()
    with torch.no_grad():
        for feature, label in test_loader:
            pred = model(feature)
            print(f"Accuracy: {accuracy_score(labels_test, pred.max(1)[1].data.numpy()) * 100}")
            loss = criterion(pred, label)
            print(f"Loss: {loss.item()}")

This will calculate the accuracy of the model against the images store in the "Test" directory and will return a loss. To run the model the function "train()" is simply run. 

This outputs

This however is flawed being that Models are significantly faster if trained on CUDA cores. I currently have an RTX 3070 with 5888 CUDA cores. Having the ability to use these for traning would significantly accelerate the training process. Taking of these cuda cores can be easily done with some configuration of the pytorch worker as seen below:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

As seen above is able, the model will train on a dedicated piece or hardware that has CUDA enabled or avaliable, however will still have the ability to train using a CPU workload.  
  
Once we try to call our train() function the following is returned:

The error encountered above is due to a mismatch in the number of samples that are being compared to the test set. This typically occurs when the model being evaluated is being evaluated based on batches of data, then directly comparing the results to a test set. This can be solved by